##### Copyright 2018 The TensorFlow Hub Authors.

##### Modifications Copyright 2019 Tomoaki Masuda.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

このノートブックは、以下のノートブックを元に日本語訳、一部章立ての再構成、加筆を行いました。 
https://github.com/tensorflow/hub/blob/master/docs/tutorials/text_classification_with_tf_hub.ipynb


#  TF-Hubでシンプルなテキスト分類器を作る方法



 TF-Hubは、再利用可能なリソース、特に事前学習済みの**モジュール**としてパッケージされた機械学習の専門知識を共有するためのプラットフォームです。このチュートリアルは、大きく分けて2つのパートから成ります。 

 **はじめに： TF-Hubを使ってテキスト分類器を学習する** 

 TF-Hubのテキストembeddingモジュールを使い、それなりのベースライン精度をもつ、シンプルな感情分類を学習します。次に予測を分析、モデルの妥当性を確認、精度改善の方法を考察します。 

 **発展：転移学習の分析** 

このセクションでは、さまざまなTF-Hubモジュールを使い、それらが推定精度に与える影響を比較、転移学習の利点と落とし穴を示します。 



## あると望ましい事前知識

必須ではないが、以下の事前知識があると、ノートブックの内容が理解しやすいです。

-  Tensorflowの[プリセット推論器フレームワーク](https://www.tensorflow.org/get_started/premade_estimators)の基本理解
-  [Pandas](https://pandas.pydata.org/)ライブラリの理解



## A. 環境を準備する

必要なライブラリのインストール、インポートを行います。


In [ ]:
# Install the latest Tensorflow version.
!pip install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip install tensorflow-hub
!pip install seaborn


 TensorFlowのインストールについて、詳しい情報は [https://www.tensorflow.org/install/](https://www.tensorflow.org/install/) にあります。 


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

# その1 - 入門編



## B. データセットを準備する

 Mass氏ほかの[Large Movie Review Dataset v1.0](http://ai.stanford.edu/~amaas/data/sentiment/)タスクに挑戦します。データセットは、1から10までのポジティブ度でラベル付けされたIMDB映画レビューです。タスクは、レビュー文から**ネガティブ評価**か**ポジティブ評価**を予測し、ラベル付けすることです。 


In [ ]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

train_df, test_df = download_and_load_datasets()
train_df.head()


## C. データセットを前処理する

整形されたデータセットのため、前処理は不要です。



## D. モデルを作成する


### 1. 入力関数

 [Estimatorフレームワーク](https://www.tensorflow.org/get_started/premade_estimators#overview_of_programming_with_estimators)はPandasのデータフレームをラップする[入力関数](https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/pandas_input_fn)を提供します。


In [ ]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.estimator.inputs.pandas_input_fn(
    train_df, train_df["polarity"], num_epochs=None, shuffle=True)

# Prediction on the whole training set.
predict_train_input_fn = tf.estimator.inputs.pandas_input_fn(
    train_df, train_df["polarity"], shuffle=False)
# Prediction on the test set.
predict_test_input_fn = tf.estimator.inputs.pandas_input_fn(
    test_df, test_df["polarity"], shuffle=False)


### 2. 特徴カラム

 TF-Hubには、与えられたテキストにモジュールを適用し、さらにそのモジュールの出力を渡す[特徴カラム](https://github.com/tensorflow/hub/blob/master/docs/api_docs/python/hub/text_embedding_column.md)があります。このチュートリアルでは、 [nnlm-en-dim128モジュール](https://tfhub.dev/google/nnlm-en-dim128/1)を使います。モジュールは以下の役割を果たします。
 
- 入力として**文のバッチを文字列の1-Dテンソル**として取る 
- **文の前処理** （句読点の削除やスペースでの単語分割など）を担当する
- どんな入力でも動作する（例えば、 **nnlm-en-dim128**は未知語（語彙にない単語）も〜2万種類にハッシュする）


In [ ]:
embedded_text_feature_column = hub.text_embedding_column(
    key="sentence", 
    module_spec="https://tfhub.dev/google/nnlm-en-dim128/1")


### 3. 推論器

分類には[DNN Classifier](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier)を使うことができます（チュートリアル最後の、ラベル関数の異なるモデル化についての注意も参照してください）。 


In [ ]:
estimator = tf.estimator.DNNClassifier(
    hidden_units=[500, 100],
    feature_columns=[embedded_text_feature_column],
    n_classes=2,
    optimizer=tf.train.AdagradOptimizer(learning_rate=0.003))


## E. モデルを学習させる

推論器を合理的なステップ数分、学習させます。 


In [ ]:
# Training for 1,000 steps means 128,000 training examples with the default
# batch size. This is roughly equivalent to 5 epochs since the training dataset
# contains 25,000 examples.
estimator.train(input_fn=train_input_fn, steps=1000);


## F. 学習済みモデルを評価する

学習データセットとテストデータセットの両方に対して予測を実行します。 


In [ ]:
train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)
test_eval_result = estimator.evaluate(input_fn=predict_test_input_fn)

print("Training set accuracy: {accuracy}".format(**train_eval_result))
print("Test set accuracy: {accuracy}".format(**test_eval_result))


### 1. 混同行列

誤分類の分布を理解するために、混同行列を見てみましょう。


In [ ]:
def get_predictions(estimator, input_fn):
  return [x["class_ids"][0] for x in estimator.predict(input_fn=input_fn)]

LABELS = [
    "negative", "positive"
]

# Create a confusion matrix on training data.
with tf.Graph().as_default():
  cm = tf.confusion_matrix(train_df["polarity"], 
                           get_predictions(estimator, predict_train_input_fn))
  with tf.Session() as session:
    cm_out = session.run(cm)

# Normalize the confusion matrix so that each row sums to 1.
cm_out = cm_out.astype(float) / cm_out.sum(axis=1)[:, np.newaxis]

sns.heatmap(cm_out, annot=True, xticklabels=LABELS, yticklabels=LABELS);
plt.xlabel("Predicted");
plt.ylabel("True");


# さらなる改良
1.  **感情の回帰** ：ここでは分類器を使い、各サンプルをポジティブな評価・ネガティブな評価という2つのクラスに割り当てました。しかし、データセット上はカテゴリー変数として「感情」という属性と、その尺度を持っています。「感情」は、0から10のスケールで表されており、基礎となる値（ポジティブ/ネガティブ）は連続した範囲にうまくマッピングできます。分類（ [DNN Classifier](https://www.tensorflow.org/api_docs/python/tf/contrib/learn/DNNClassifier) ）の代わりに回帰（ [DNN Regressor](https://www.tensorflow.org/api_docs/python/tf/contrib/learn/DNNRegressor) ）を計算すると、この0から10のスケールを利用できます。 
1.  **大きいモジュール** ：このチュートリアルでは、メモリ使用量を抑えるために小さいモジュールを使いました。より大きな語彙・埋め込み空間（単語の分散表現の次元）を持つモジュールがあります。それらを使うと、精度向上が見込まれます。
1.  **パラメータ調整** ：特に別のモジュールを使う場合、学習率やステップ数などのハイパーパラメータを調整し、精度を向上できます。妥当な結果を得るには、検証データセットがとても大事です。何もしないと、モデルは学習データセットだけが得意な（過学習した）状態に、簡単に陥ってしまうからです。 
1.  **より複雑なモデル** ：今回は、各文のembeddingの計算に、各文が持つ各単語のembeddingを平均するモジュールを使いました。文の性質をよりよく捉えるため、シーケンシャルモジュール（例： [Universal Sentence Encoder](https://tfhub.dev/google/universal-sentence-encoder/2)モジュール）も使えます。または2つ以上のTF-Hubモジュールのアンサンブルもできます。
1.  **正則化** ：過学習を防ぐため、正則化を行うオプティマイザを使えます。例えば[Proximal Adagrad Optimizer](https://www.tensorflow.org/api_docs/python/tf/train/ProximalAdagradOptimizer)などが挙げられます。



# その2 - 発展：転移学習の分析

転移学習により、 **学習リソースを節約**し、**小規模なデータセットで学習**して、優れた汎化性能を達成できます。このパートでは、2つの異なるTF-Hubモジュールを使った学習で、これを実証します。 

-  **[nnlm-en-dim128](https://tfhub.dev/google/nnlm-en-dim128/1)** - 事前学習済みのテキストembeddingモジュール
-  **[random-nnlm-en-dim128](https://tfhub.dev/google/random-nnlm-en-dim128/1)** - **nnlm-en-dim128**と同じ語彙やネットワークを持つテキストembeddingモジュール。ただし重みはランダムに初期化され、実際のデータでの学習は行なっていません 



## E. モデルを学習させる

これらを2つのモードで学習させます。 

-  **分類器のみを**学習させる （つまりモジュールの重みは変えない（凍結する））
-  **モジュールと分類器を共に**学習させる 

いくつかの学習と評価を試し、モジュールごとに、精度にどんな違いが出るか確認しましょう。 



In [ ]:
def train_and_evaluate_with_module(hub_module, train_module=False):
  embedded_text_feature_column = hub.text_embedding_column(
      key="sentence", module_spec=hub_module, trainable=train_module)

  estimator = tf.estimator.DNNClassifier(
      hidden_units=[500, 100],
      feature_columns=[embedded_text_feature_column],
      n_classes=2,
      optimizer=tf.train.AdagradOptimizer(learning_rate=0.003))

  estimator.train(input_fn=train_input_fn, steps=1000)

  train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)
  test_eval_result = estimator.evaluate(input_fn=predict_test_input_fn)

  training_set_accuracy = train_eval_result["accuracy"]
  test_set_accuracy = test_eval_result["accuracy"]

  return {
      "Training accuracy": training_set_accuracy,
      "Test accuracy": test_set_accuracy
  }


results = {}
results["nnlm-en-dim128"] = train_and_evaluate_with_module(
    "https://tfhub.dev/google/nnlm-en-dim128/1")
results["nnlm-en-dim128-with-module-training"] = train_and_evaluate_with_module(
    "https://tfhub.dev/google/nnlm-en-dim128/1", True)
results["random-nnlm-en-dim128"] = train_and_evaluate_with_module(
    "https://tfhub.dev/google/random-nnlm-en-dim128/1")
results["random-nnlm-en-dim128-with-module-training"] = train_and_evaluate_with_module(
    "https://tfhub.dev/google/random-nnlm-en-dim128/1", True)

## F. 学習済みモデルを評価する


結果を見てみましょう。 


In [ ]:
pd.DataFrame.from_dict(results, orient="index")

We can already see some patterns, but first we should establish the baseline accuracy of the test set - the lower bound that can be achieved by outputting only the label of the most represented class:


すでに一定の傾向が見えますが、まずテストデータのベースラインの精度を確認します。ベースラインの精度とは、最も出現頻度の高いクラスのラベルで全出力を埋めると仮定すると出る、精度の下限です。 


In [ ]:
estimator.evaluate(input_fn=predict_test_input_fn)["accuracy_baseline"]


最も出現頻度の高いクラスで埋めると、 **50％** という精度が得られます。ここでいくつか注意があります。 

1. 不思議なことに、**固定かつランダムなembedding上でも、モデルの学習は進みます。** その理由は、たとえ辞書内のすべての単語がランダムなベクトルに対応づけられていても、推論器は、全結合層を使って、空間を分離できるからです。 
1.  **ランダムなembedding**のモジュールを学習可とすると、分類器だけの学習のに比べ、学習、テスト両方の精度向上がみられます。
1.  **学習済みembeddingモジュール**を学習しても、学習、テスト両方の精度が向上します。ただし学習セットへの過学習に注意してください。事前学習モジュールの追加学習は、正則化を使ってもなお危険です。もともと事前学習時は多様なデータセットで言語モデルが獲得されていましたが、追加学習で、新しいデータセットという局所での最適な表現に収束してしまいます。